# SCaling and Queues


## reference

https://kubernetes.io/docs/tasks/job/fine-parallel-processing-work-queue/


In this example, we will run a Kubernetes Job with multiple parallel worker processes in a given pod.

In this example, as each pod is created, it picks up one unit of work from a task queue, processes it, and repeats until the end of the queue is reached.


1. Start a storage service to hold the work queue.
2. Create a queue, and fill it with messages. 
3. Start a Job that works on tasks from the queue. 


In [1]:
%env k=microk8s.kubectl

env: k=microk8s.kubectl


In [2]:
! $k get all

NAME                 TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.152.183.1   <none>        443/TCP   23m


In [4]:
! microk8s.status |grep dns

dns: enabled


In [5]:
! mkdir -p redis

In [6]:
! (cd redis; wget https://kubernetes.io/examples/application/job/redis/redis-pod.yaml)

--2020-03-30 14:22:06--  https://kubernetes.io/examples/application/job/redis/redis-pod.yaml
Resolving kubernetes.io (kubernetes.io)... 147.75.40.148
Connecting to kubernetes.io (kubernetes.io)|147.75.40.148|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 240 [application/x-yaml]
Saving to: ‘redis-pod.yaml’

redis-pod.yaml      100%[===================>]     240  --.-KB/s    in 0s      

2020-03-30 14:22:07 (7.74 MB/s) - ‘redis-pod.yaml’ saved [240/240]



In [8]:
! ls -l redis

total 4
-rw-rw-r-- 1 ubuntu ubuntu 240 Mar 30 14:22 redis-pod.yaml


In [9]:
%%bash

cd redis
wget https://kubernetes.io/examples/application/job/redis/redis-service.yaml
wget https://kubernetes.io/examples/application/job/redis/Dockerfile
wget https://kubernetes.io/examples/application/job/redis/job.yaml
wget https://kubernetes.io/examples/application/job/redis/rediswq.py
wget https://kubernetes.io/examples/application/job/redis/worker.py

--2020-03-30 14:23:58--  https://kubernetes.io/examples/application/job/redis/redis-service.yaml
Resolving kubernetes.io (kubernetes.io)... 147.75.40.148
Connecting to kubernetes.io (kubernetes.io)|147.75.40.148|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 135 [application/x-yaml]
Saving to: ‘redis-service.yaml’

     0K                                                       100% 6.59M=0s

2020-03-30 14:23:59 (6.59 MB/s) - ‘redis-service.yaml’ saved [135/135]

--2020-03-30 14:23:59--  https://kubernetes.io/examples/application/job/redis/Dockerfile
Resolving kubernetes.io (kubernetes.io)... 147.75.40.148
Connecting to kubernetes.io (kubernetes.io)|147.75.40.148|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 115 [text/plain]
Saving to: ‘Dockerfile’

     0K                                                       100% 5.87M=0s

2020-03-30 14:23:59 (5.87 MB/s) - ‘Dockerfile’ saved [115/115]

--2020-03-30 14:23:59--  https://kubernetes.io

In [10]:
! ls -l redis

total 28
-rw-rw-r-- 1 ubuntu ubuntu  115 Mar 30 14:23 Dockerfile
-rw-rw-r-- 1 ubuntu ubuntu  244 Mar 30 14:23 job.yaml
-rw-rw-r-- 1 ubuntu ubuntu  240 Mar 30 14:22 redis-pod.yaml
-rw-rw-r-- 1 ubuntu ubuntu  135 Mar 30 14:23 redis-service.yaml
-rw-rw-r-- 1 ubuntu ubuntu 5545 Mar 30 14:24 rediswq.py
-rw-rw-r-- 1 ubuntu ubuntu  657 Mar 30 14:24 worker.py


# Run the redis cli interactively

in a terminal windo do this

## kubectl run -i --tty temp --image redis --command "/bin/sh"

# OOPS - need a basic redis service up

In [11]:
! $k create -f ./redis/redis-pod.yaml

pod/redis-master created


In [12]:
! $k create -f ./redis/redis-service.yaml

service/redis created


In [13]:
! $k create -f ./redis/job.yaml

job.batch/job-wq-2 created


In [14]:
! $k get pods

NAME                    READY   STATUS         RESTARTS   AGE
job-wq-2-pzmc6          0/1     ErrImagePull   0          9s
job-wq-2-qxmhb          0/1     ErrImagePull   0          9s
redis-master            1/1     Running        0          9m5s
temp-5878477f6c-zxqvx   1/1     Running        0          11m


In [15]:
! $k describe jobs/job-wq-2

Name:           job-wq-2
Namespace:      default
Selector:       controller-uid=9182c9c5-97f9-44c9-89b0-cb9fcf67fdf8
Labels:         controller-uid=9182c9c5-97f9-44c9-89b0-cb9fcf67fdf8
                job-name=job-wq-2
Annotations:    <none>
Parallelism:    2
Completions:    <unset>
Start Time:     Mon, 30 Mar 2020 14:39:39 +0000
Pods Statuses:  2 Running / 0 Succeeded / 0 Failed
Pod Template:
  Labels:  controller-uid=9182c9c5-97f9-44c9-89b0-cb9fcf67fdf8
           job-name=job-wq-2
  Containers:
   c:
    Image:        gcr.io/myproject/job-wq-2
    Port:         <none>
    Host Port:    <none>
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Events:
  Type    Reason            Age   From            Message
  ----    ------            ----  ----            -------
  Normal  SuccessfulCreate  48s   job-controller  Created pod: job-wq-2-qxmhb
  Normal  SuccessfulCreate  48s   job-controller  Created pod: job-wq-2-pzmc6


In [17]:
!$k logs pods/job-wq-2-pzmc6

Error from server (BadRequest): container "c" in pod "job-wq-2-pzmc6" is waiting to start: trying and failing to pull image
